Dans ce notebook, il s'agira de faire la partie modélisation du projet. Après un nettoyage de la base des critiques en utilisant des techniques de NLP, nous allons réliaser des modèles permettant de prédire la positivité ou négativité de la critique. Ensuite, nous comparerons les résultats obtenus à un modèle pré entrainé sur des textes en français, issue de BERT, qu'on aura au préalable fit sur nos données.

### Imports nécessaires

In [1]:
# Classique python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import seaborn as sns


In [2]:
# NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [16]:
# ML

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing, naive_bayes, metrics


In [4]:
# CamemBERT
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset


/home/onyxia/work/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Nettoyage + NLP

In [ ]:
# Lecture du df enregistré après le scraping des critiques sur Allociné
df_critiques = pd.read_csv("df_critiques.csv")

In [7]:
df_critiques = df_critiques.dropna(subset=['Critique'])

On va commencer par convertir les notes en float et extraire la date.

In [ ]:
# Convertir les notes en float
df_critiques['Note_critique_float'] = df_critiques['Note de la critique'].str.replace(',', '.').astype(float)

# Extraire la date
df_critiques['new_date1'] = df_critiques['Date de publication'].str.replace('Publiée le', '').apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_critiques['new_date'] = df_critiques['new_date1'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)


colonnes_a_supprimer = ["new_date1", "Date de publication",'Note de la critique']
df_critiques = df_critiques.drop(columns=colonnes_a_supprimer)

In [ ]:
# Rôle de la fonction : Définir une polarité à partir des notes des commentaires
# Entrée : note d'un commentaire
# Sortie : Retourne une polarité suivant la note

def find_polarity(note_commentaire):
    if float(note_commentaire) <= 3:
        return 0 # Négatif
    else:
        return 1 # Positif

In [ ]:
df_critiques['Polarité_réelle'] = df_critiques['Note_critique_float'].apply(find_polarity) 

In [ ]:
# Rôle de la fonction : Nettoyer les critiques dans le but de faire du NLP
# Entrée : Critique du df
# Sortie : Renvoie la critique nettoyé


def clean_text_french(critique):
    # Assurez-vous que la colonne de commentaire est de type chaîne de caractères
    critique = critique.astype(str)

    # Convertir le texte en minuscules
    critique = critique.str.lower()

    # Tokenisation des mots
    critique = critique.apply(word_tokenize, language='french')

    # Supprimer la ponctuation et les caractères spéciaux
    critique = critique.apply(lambda tokens: [word for word in tokens if word.isalnum()])

    # Supprimer les mots vides
    stop_words = set(stopwords.words('french'))
    critique = critique.apply(lambda tokens: [word for word in tokens if word not in stop_words])

    # Racinisation (stemming)
    lemmatizer = WordNetLemmatizer()
    critique = critique.apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

    # Rejoindre les tokens en une seule chaîne de texte
    critique = critique.apply(lambda tokens: ' '.join(tokens))


    return critique

# python -m nltk.downloader all

In [ ]:
df_critiques = df_critiques[df_critiques['Critique'].astype(str).apply(lambda x: isinstance(x, str))]

In [ ]:
%%time
df_critiques['Critique_nettoye'] = pd.DataFrame(clean_text_french(df_critiques['Critique']))

# CPU times: user 20min, sys: 16.5 s, total: 20min 17s
# Wall time: 20min 17s

In [ ]:
# On fait attention aux valeurs nulles
df_critiques = df_critiques.dropna(subset=['Critique_nettoye'])

In [ ]:
# Rôle : 
# Entrée : 
# Sortie : 


def compter_mots(critique):
    if isinstance(critique, str):
        mots = critique.split()
        return len(mots)
    else:
        # Si la valeur n'est pas une chaîne de caractères, vous pouvez choisir de traiter cela d'une manière particulière.
        # Par exemple, vous pourriez retourner 0 ou une autre valeur appropriée.
        return 0

In [ ]:
# Appliquer la fonction pour compter les mots à la colonne 'Critique' et créer une nouvelle colonne 'nb_mots'
df_critiques['nb_mots'] = df_critiques['Critique'].apply(compter_mots)

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())


# Rôle de la fonction : Utilisation de text blob pour atrrtibuer une polarité (postitive ou négative) à un commentaire
# Entrée : Commentaire
# Sortie : Retourne une polarité du commentaire

def attribuer_note(commentaire):
    #
    vs = tb(commentaire).sentiment[0]
    if vs > 0:
        return 1
    else:
        return 0

In [ ]:
%%time

# Appliquer la fonction pour attribuer une note à chaque commentaire
df_critiques['Text_blob_critique_nettoyé'] = df_critiques['Critique_nettoye'].apply(attribuer_note)

# CPU times: user 3min 46s, sys: 188 ms, total: 3min 46s
# Wall time: 3min 46s

In [ ]:
df_critiques

In [ ]:
df_critiques.to_csv("df_critiques_modif.csv", index=False)

### Machine Learning

In [6]:
df_critiques = pd.read_csv("df_critiques_modif.csv")

In [7]:
df_critiques = df_critiques.dropna(subset=['Critique_nettoy'])

In [8]:
df_critiques = df_critiques[df_critiques['nb_mots'] <= 300]

In [70]:
df_critiques

,id_allocine,Critique,Note_critique_float,new_date,Polarité_réelle,Critique_nettoy,nb_mots,Text_blob_critique_nettoyé
0,239331,"""10.000 days"" est à la base une série créée pa...",0.5,2015-10-05,0,day base série créée eric small réalise film o...,129,0
1,239331,Les effets spéciaux bas de gamme mettent les a...,1.0,2015-12-12,0,effets spéciaux ba gamme mettent acteurs situa...,18,0
2,239331,"10,000 somnifères. A peine ça commence qu'un g...",1.0,2017-01-10,0,somnifères a peine ça commence gar raconte lif...,45,0
3,239331,"""10 000 days"" est d'une nullité abyssale. Tout...",0.5,2016-05-24,0,10 000 day nullité abyssale tout concourt fair...,84,0
4,239331,"Beaucoup de parlotes, pas d'action ni d'effets...",1.5,2017-01-01,0,beaucoup parlotes ni spéciaux histoire molle t...,22,0
...,...,...,...,...,...,...,...,...
801583,146631,"J'avais beaucoup aimé le premier Zoolander, fo...",2.0,2016-05-21,0,beaucoup aimé premier zoolander fort quantité ...,115,1
801585,146631,"La sauce ne prend malheureusement pas. ""Zoolan...",2.0,2016-05-17,0,sauce prend malheureusement zoolander devenu c...,72,0
801586,146631,la demi étoiles et pour le casting avec des ac...,0.5,2020-07-15,0,demi étoiles casting acteurs fou film a humour...,43,0
801587,146631,Pas aussi mauvais...Pas très Drôle mais amusan...,2.5,2016-03-21,0,aussi mauvais très drôle amusant bien filmé te...,46,1


In [9]:
df_critiques_mod = df_critiques.sample(n=100000, random_state=2)  # Utilisez un random_state pour la reproductibilité

# 40 echantillons à 100 000

In [10]:
df_critiques_mod.groupby('Polarité_réelle').size()


Polarité_réelle
0    45740
1    54260
dtype: int64

In [11]:
df_critiques_mod

,id_allocine,Critique,Note_critique_float,new_date,Polarité_réelle,Critique_nettoy,nb_mots,Text_blob_critique_nettoyé
50107,237763,Film monotone et sans surprise qui se contente...,0.5,2018-03-17,0,film monotone sans surprise contente faire pub...,89,0
501267,229359,"À l'image de son personnage : sans envergure, ...",1.5,2016-11-13,0,personnage sans envergure naïf film rapidement...,93,1
262765,114782,"un vrai bonheur visuelun vrai drame, humain, p...",5.0,2018-05-29,1,vrai bonheur visuelun vrai drame humain person...,54,1
380156,239713,J'y suis allée un peu au hasard d'un horaire d...,4.0,2017-01-23,1,allée peu hasard horaire ciné excellente surpr...,55,1
329420,229070,Avec ses 6 nominations aux Oscars et sa promot...,4.0,2018-02-05,1,6 nomination oscar promotion basée histoire vr...,170,1
...,...,...,...,...,...,...,...,...
679484,144195,"Very Bad Cops : Je ne comprends pas, a mon pre...",5.0,2012-05-08,1,very bad cop comprends a premier visionnage fi...,276,1
657630,179426,"Salut ,Amateur de films épouvante/horreur , j'...",1.0,2010-09-18,0,salut amateur film très déçu celui ci première...,76,0
775080,182075,J'étais impatient de voir le film au vu de son...,1.5,2012-04-16,0,impatient voir film vu déception film constamm...,105,0
459589,265585,"On a bien la patte ""Zemeckis"" avec les effets ...",2.0,2020-12-29,0,a bien patte zemeckis effets réalisation musiq...,43,1


In [12]:
# On prépare nos  jeux de test et d'entrainement pour le countvectorizer et pour le ML

critique_train, critique_test, polarite_train, polarite_test = train_test_split(df_critiques_mod['Critique_nettoy'], df_critiques_mod['Polarité_réelle'], test_size=0.2, random_state=4)

In [13]:
# CountVectorizer 
# Check limit colonne

cv = CountVectorizer()
cv_train_features = cv.fit_transform(critique_train)
cv_test_features = cv.transform(critique_test)

In [14]:
# Rôle de la fonction : Fit + résultat du modèle
# Entrée : Modèle considéré, les 4 sets de test et d'entrainement
# Sortie : les résultats du modèles
from sklearnex import patch_sklearn
patch_sklearn()


def fitting_predictions(model, train_features, train_labels, test_features):
    model.fit(train_features, train_labels)
    predictions = model.predict(test_features)
    return predictions

def print_metrics(test_labels, predictions):
    # On stock nos données dans un dictionnaire
    results = {
        'Accuracy': metrics.accuracy_score(test_labels, predictions),
        'Precision': metrics.precision_score(test_labels, predictions),
        'Recall': metrics.recall_score(test_labels, predictions),
        'F1 Score': metrics.f1_score(test_labels, predictions),
        'Classification Report': metrics.classification_report(test_labels, predictions),
        'Confusion Matrix': metrics.confusion_matrix(test_labels, predictions)
    }
    
    # Print metriques
    print("\nAccuracy:", results['Accuracy'])
    print("Precision:", results['Precision'])
    print("Recall:", results['Recall'])
    print("F1 Score:", results['F1 Score'])
    
    # Print classification report
    print("\nClassification Report:")
    print(results['Classification Report'])

    # Display confusion matrix as a heatmap using seaborn
    cm = confusion_matrix(test_labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    return results


def evaluate_model_final(model, train_features, train_labels, test_features, test_labels):
    
    predictions = fitting_predictions(model, train_features, train_labels, test_features)

    # Cross-validation 
    cv_scores = cross_val_score(model, train_features, train_labels, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
    print("Cross-validation Scores:", cv_scores)
    print("Mean CV Accuracy:", np.mean(cv_scores))

    print_metrics(test_labels, predictions)

    return 1

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


#### Résultats Text Blob

In [17]:
test_labels= df_critiques['Polarité_réelle']
predictions = df_critiques['Text_blob_critique_nettoyé']

print_metrics(test_labels, predictions)



Accuracy: 0.6695942535119721
Precision: 0.6363074169415172
Recall: 0.924393337731995
F1 Score: 0.7537618409023455

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.36      0.50    340058
           1       0.64      0.92      0.75    410731

    accuracy                           0.67    750789
   macro avg       0.72      0.64      0.63    750789
weighted avg       0.71      0.67      0.64    750789



NameError: name 'confusion_matrix' is not defined

### Modèles ML

In [ ]:
%%time
model_lr = LogisticRegression(C=0.5, max_iter=100000)
logistic_regression_results = evaluate_model_final(model_lr, cv_train_features, polarite_train, cv_test_features, polarite_test)

In [ ]:
%%time
# Example with Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_results = evaluate_model_final(random_forest, cv_train_features, polarite_train, cv_test_features, polarite_test)

In [ ]:
%%time
# Example with Support Vector Machine
svm = SVC(kernel='linear', C=1)
svm_results = evaluate_model_final(svm, cv_train_features, polarite_train, cv_test_features, polarite_test)

In [ ]:
%%time
# Naive Bayes on Count Vectors

model_MultinomialNB= naive_bayes.MultinomialNB()
MultinomialNB_results = evaluate_model_final(model_MultinomialNB, cv_train_features, polarite_train, cv_test_features, polarite_test)

## CamemBERT

In [18]:
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [19]:
df_critiques_mod

,id_allocine,Critique,Note_critique_float,new_date,Polarité_réelle,Critique_nettoy,nb_mots,Text_blob_critique_nettoyé
50107,237763,Film monotone et sans surprise qui se contente...,0.5,2018-03-17,0,film monotone sans surprise contente faire pub...,89,0
501267,229359,"À l'image de son personnage : sans envergure, ...",1.5,2016-11-13,0,personnage sans envergure naïf film rapidement...,93,1
262765,114782,"un vrai bonheur visuelun vrai drame, humain, p...",5.0,2018-05-29,1,vrai bonheur visuelun vrai drame humain person...,54,1
380156,239713,J'y suis allée un peu au hasard d'un horaire d...,4.0,2017-01-23,1,allée peu hasard horaire ciné excellente surpr...,55,1
329420,229070,Avec ses 6 nominations aux Oscars et sa promot...,4.0,2018-02-05,1,6 nomination oscar promotion basée histoire vr...,170,1
...,...,...,...,...,...,...,...,...
679484,144195,"Very Bad Cops : Je ne comprends pas, a mon pre...",5.0,2012-05-08,1,very bad cop comprends a premier visionnage fi...,276,1
657630,179426,"Salut ,Amateur de films épouvante/horreur , j'...",1.0,2010-09-18,0,salut amateur film très déçu celui ci première...,76,0
775080,182075,J'étais impatient de voir le film au vu de son...,1.5,2012-04-16,0,impatient voir film vu déception film constamm...,105,0
459589,265585,"On a bien la patte ""Zemeckis"" avec les effets ...",2.0,2020-12-29,0,a bien patte zemeckis effets réalisation musiq...,43,1


In [20]:
critique = df_critiques_mod['Critique'].values.tolist()
polarite = df_critiques_mod['Polarité_réelle'].values.tolist()

print(len(critique))
print(len(polarite))


100000
100000


In [21]:
# Séparer les données en ensembles d'entraînement et de test
split_border = int(len(polarite)*0.8)
critique_train, critique_test = critique[:split_border], critique[split_border:]
polarite_train, polarite_test = polarite[:split_border], polarite[split_border:]

# Charger le modèle Camembert pré-entraîné et le tokenizer
model_name = "camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = CamembertForSequenceClassification.from_pretrained(model_name)

print(len(critique_train))
print(len(polarite_train))

print(len(critique_test))
print(len(polarite_test))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


80000
80000
20000
20000


In [22]:
# Prétraiter les données
def preprocess_data(critique, polarite, tokenizer):
    encoded = tokenizer.batch_encode_plus(
        critique,
        add_special_tokens=False,
        padding=True,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    polarite_ = torch.tensor(polarite)

    return encoded['input_ids'], encoded['attention_mask'], polarite_

    #torch.tensor(polarite_onehot, dtype=torch.float32)


In [23]:
input_ids_train, attention_mask_train, polarite_train = preprocess_data(critique_train, polarite_train, tokenizer)
print(input_ids_train.shape)
print(attention_mask_train.shape)
print(polarite_train.shape)
#train_dataset = TensorDataset(input_ids_train, attention_mask, polarite)


input_ids_test, attention_mask_test, polarite_test = preprocess_data(critique_test, polarite_test, tokenizer)
print(input_ids_test.shape)
print(attention_mask_test.shape)
print(polarite_test.shape)
#test_dataset = TensorDataset(input_ids, attention_mask, polarite)


torch.Size([80000, 512])
torch.Size([80000, 512])
torch.Size([80000])
torch.Size([20000, 497])
torch.Size([20000, 497])
torch.Size([20000])


In [25]:
# Déplacez le modèle sur le GPU s'il est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
model = model.to(device)

# Déplacez les données d'entraînement sur le GPU
input_ids_train_, attention_mask_train_, polarite_train_ = input_ids_train.to(device), attention_mask_train.to(device), polarite_train.to(device)
train_dataset = TensorDataset(input_ids_train_, attention_mask_train_, polarite_train_)

# Déplacez les données de test sur le GPU
input_ids_test_, attention_mask_test_, polarite_test_ = input_ids_test.to(device), attention_mask_test.to(device), polarite_test.to(device)
test_dataset = TensorDataset(input_ids_test_ , attention_mask_test_, polarite_test_)


# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# Define training parameters
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


In [26]:
from tqdm import tqdm
import time

model.train()
for epoch in range(epochs):
    total_loss = 0.0
    num_batches = len(train_loader)

    # Initialize tqdm for progress bar
    progress_bar = tqdm(enumerate(train_loader), total=num_batches, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch')

    for batch_idx, batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels.unsqueeze(1))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update tqdm bar with current loss
        progress_bar.set_postfix({'loss': loss.item(), 'avg_loss': total_loss / (batch_idx + 1)})

        # Optional: Simulate time delay to show the progress bar
        time.sleep(0.1)

    # Print epoch-level training statistics
    avg_epoch_loss = total_loss / num_batches
    print(f'Epoch [{epoch + 1}/{epochs}], Average Training Loss: {avg_epoch_loss:.4f}')

# Additional code for model evaluation or saving the trained model can be added here


Epoch 1/5:   0%|          | 0/2500 [00:01<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 14.54 GiB of which 2.75 MiB is free. Process 3997949 has 14.53 GiB memory in use. Of the allocated memory 14.37 GiB is allocated by PyTorch, and 94.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [127]:
torch.cuda.empty_cache()


In [ ]:
# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(logits.argmax(dim=1).tolist())
        true_labels.extend(labels.squeeze().tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
print_metrics(true_labels, predictions)